In [28]:
import cv2
import numpy as np
import os

def ssim(img1, img2):
    # Ensure both images are in float format for calculations
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    
    # Set the constants for SSIM (these are standard values)
    C1 = (0.01 * 255) ** 2
    C2 = (0.03 * 255) ** 2
    
    # Compute means
    mu1 = cv2.GaussianBlur(img1, (11, 11), 1.5)
    mu2 = cv2.GaussianBlur(img2, (11, 11), 1.5)
    
    # Compute variances and covariance
    sigma1_sq = cv2.GaussianBlur(img1 ** 2, (11, 11), 1.5) - mu1 ** 2
    sigma2_sq = cv2.GaussianBlur(img2 ** 2, (11, 11), 1.5) - mu2 ** 2
    sigma12 = cv2.GaussianBlur(img1 * img2, (11, 11), 1.5) - mu1 * mu2
    
    # Compute SSIM
    ssim_map = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / ((mu1 ** 2 + mu2 ** 2 + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

def calculate_hu_moments(image):
    # Calculate Hu Moments which are invariant to scale, rotation, and reflection
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()
    return hu_moments

# Load the image
image = cv2.imread('network.png')

# Convert the image to HSV color space for better color segmentation
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define a broader range for blue color
lower_blue = np.array([90, 50, 50])  # Adjusted lower bound for a wider range
upper_blue = np.array([140, 255, 255])  # Keep the upper bound for blue

# Create a mask for blue areas in the image
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Apply morphological operations to clean up the mask
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

template_dir = 'node_images'  # Change this to your actual path
templates_hu_moments = {}
for filename in os.listdir(template_dir):
    template_path = os.path.join(template_dir, filename)
    template_img = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    if template_img is not None:
        # Calculate Hu Moments for each template and store
        templates_hu_moments[filename] = calculate_hu_moments(template_img)

# Loop through each contour and identify the closest matching icon based on Hu Moments
for contour in contours:
    # Get bounding box for each contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Extract the region of interest (ROI) from the original image
    roi = image[y:y+h, x:x+w]
    
    # Convert the ROI to grayscale
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    # Calculate Hu Moments for the ROI
    roi_hu_moments = calculate_hu_moments(roi_gray)
    
    best_match_name = None
    best_match_score = float('inf')  # Initialize with a high score
    
    # Compare Hu Moments of ROI with each template
    for name, template_hu_moments in templates_hu_moments.items():
        # Use Euclidean distance to compare Hu moments
        score = np.linalg.norm(roi_hu_moments - template_hu_moments)
        
        # Update the best match if the score is lower
        if score < best_match_score:
            best_match_score = score
            best_match_name = name
    
    if best_match_name == "switch.png":
        center_x, center_y = w // 2, h // 2
        center_intensity = roi_gray[center_y, center_x]

        # If center is bright (white), confirm it as a switch
        if center_intensity > 200:  # 200 as threshold for white
            best_match_name = "switch.png"
        else:
            best_match_name = "computer.png"

    # Draw the square bounding box around the detected icon
    side_length = max(w, h)
    x = x + w // 2 - side_length // 2
    y = y + h // 2 - side_length // 2
    cv2.rectangle(image, (x, y), (x + side_length, y + side_length), (0, 255, 0), 2)
    
    # Create the label text with the best match name and Hu Moments score
    label_text = f"{best_match_name}"
    
    # Position text beside the box (right side of the box)
    text_position = (x + side_length + 10, y + 20)
    
    # Add a filled rectangle as background for the text to improve readability
    (text_width, text_height), baseline = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
    cv2.rectangle(image, (text_position[0] - 5, text_position[1] - text_height - 5), 
                  (text_position[0] + text_width + 5, text_position[1] + baseline - 5), 
                  (0, 0, 0), cv2.FILLED)
    
    # Draw the label text on top of the black rectangle
    cv2.putText(image, label_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Save the output image
output_path = 'boxed.png'
cv2.imwrite(output_path, image)
print(f"Labeled network diagram saved at {output_path}")

Labeled network diagram saved at boxed.png
